IMPORTS

In [9]:
# Imports
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, asdict
from typing import Any, Dict, List, Optional, Tuple
from deap import base, creator, tools

# File name
FILENAME = "tsp.dat"

HELPER FUNCTIONS

In [10]:
def haversine(lat_p1, long_p1, lat_p2, long_p2):
    """
    Function calculates the haversine distance between 2 points.
    Refer to this website regarding calculation: https://en.wikipedia.org/wiki/Haversine_formula
    """
    Earth_radius = 3958.88      # miles
    # Convert to radian for math. functions
    phi_1 = math.radians(lat_p1)
    phi_2 = math.radians(lat_p2)
    lambda_p1 = math.radians(long_p1)
    lambda_p2 = math.radians(long_p2)

    dphi = phi_2 - phi_1
    dlambda = lambda_p2 - lambda_p1

    hav_dphi = math.pow(math.sin(dphi/2), 2)
    cos_phi1 = math.cos(phi_1)
    cos_phi2 = math.cos(phi_2)
    hav_dlambda = math.pow(math.sin(dlambda/2), 2)

    hav_theta = hav_dphi + cos_phi1*cos_phi2*hav_dlambda
    theta = 2 * math.asin(math.sqrt(hav_theta))

    return theta * Earth_radius

def haversine_matrix(coordinates):
    """
    This function will calculate the haversine distance between all pair of coordinates, that way eval(ind) will not need to
    coordinates: k points of {latitude, longitude}
    """
    k = len(coordinates)

    # Create the K*K matrix and init with 0s
    hav_matrix = np.zeros((k, k))

    # compute the haversine distance for each k
    for i in range(k):
        lat_1, long_1 = coordinates[i]      # CityPoint 1

        for j in range(i + 1, k):
            lat_2, long_2 = coordinates[j]      # CityPoint 2 through k

            haversine_distance = haversine(lat_1, long_1, lat_2, long_2)

            # based off calc, matrix should be symmetric i.e D[i][j] = D[j][i]
            # Diagional i.e D[i][i] always = 0
            # Ex: Distance from NYC to Paris == Distance from Paris to NYC
            hav_matrix[i][j] = haversine_distance
            hav_matrix[j][i] = haversine_distance

    return hav_matrix

def evaluate_TSP(Haversine_Dist_matrix):
    """
    We need to take in the haversine_matrix as it holds the precomputed haversine distances, 
    from here we evaluate the permutations of city-point indices.
    We then need to walk through the list of the permutation, 
    adding the distances from the current city to the adjacent city.
    """
    def eval(ind):
        total_dist = 0.0

        # Walk through the permutation
        for i in range(len(ind) - 1):
            cur_city = ind[i]
            adjacent_city = ind[i + 1]

            total_dist += Haversine_Dist_matrix[cur_city][adjacent_city]
        
        # Complete the cycle
        total_dist += Haversine_Dist_matrix[ind[-1]][ind[0]]
        return (total_dist,)  # DEAP expects tuple
        
    return eval

def compare_with_best(best_min_distance):
    # Best minimum distance is 10,637.36 miles
    optimum = 10637.36
    # Compare our best with the true best and represent as a %
    percent = ((best_min_distance - optimum) / optimum ) * 100
    return percent

def load_dat(file):
    cities = []
    # Latitude Longitude Coordinates
    coordinates = []

    with open(file, "r") as f:
        for capital_cities in f:
            # Extract information
            parts = capital_cities.strip().split()

            # FORMAT: Albany, NY          42.652552778 -73.75732222
            city_state = " ".join(parts[:-2]) 
            latitude = float(parts[-2])
            longitude = float(parts[-1])

            # Append to list
            cities.append(city_state)
            coordinates.append((latitude, longitude))

    return cities, coordinates



FUNCTION TESTER

In [11]:
# cities, coordinates = load_dat(FILENAME)
# print(haversine_matrix(coordinates))   # Verified haversine distance for first 3 coordinates 

Genetic Algorithm (GA) Configuration and Implementation

In [12]:
# DEAP setup
def ensure_deap_creators() -> None:
    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))     # We are MINIMIZING
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)


def mutInsert(individual):
    size = len(individual)
    i, j = sorted(random.sample(range(size), 2))

    gene = individual.pop(j)
    individual.insert(i, gene)

    return (individual,)
        
# ===================== GA Configuration Dataclass =====================
# Configuration dataclass
@dataclass(frozen=True)         # Parameters cant be changed during runs
class Config:
    genome_length: int = 49     # 49 indices of city, cooridnates
    pop_size: int = 1500        # n individuals
    generations: int = 1000     # Allowed generations
    cxpb: float = 0.7           # crossover prob
    mutpb: float = 0.19         # mutation probability
    tournsize: int = 3          # For tournament slection
    eliteSize: int = 1          # Best k individuals to transfer to next gen
    seed: Optional[int] = None

# Build DEAP toolbox
def build_toolbox(cfg: Config) -> base.Toolbox:
    ensure_deap_creators()

    toolbox = base.Toolbox()
    # EX of ind: [1, 2, 3, 4......49]  representing city1 --> city2-->city49
    toolbox.register("indices", random.sample, range(cfg.genome_length), cfg.genome_length) # Creates the list of indices reppresenting cities 1-49
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutInversion)
    toolbox.register("select", tools.selTournament, tournsize=cfg.tournsize)

    return toolbox


# Single run of the GA
def run(cfg: Config) -> Dict[str, Any]:
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)

    # List of city names and coordinates
    cities, coordinates = load_dat(FILENAME)

    # Fill the Haversine matrix with the Haversine distances for each City-Point
    Haversine_Dist_matrix = haversine_matrix(coordinates)

    # Setup toolbox
    toolbox = build_toolbox(cfg)
    toolbox.register("evaluate", evaluate_TSP(Haversine_Dist_matrix))
    pop = toolbox.population(n=cfg.pop_size)    # Create initial popultation

    # Evaluate initial population (gen=0)
    fitnesses = map(toolbox.evaluate, pop)  # Compute fitness for each ind
    for ind, fit in zip(pop, fitnesses):    # Assign fitness back to ind
        ind.fitness.values = fit

    # Records Global Best solution, so that it is not lost
    hall_of_fame = tools.HallOfFame(cfg.eliteSize)
    
    # Link current statistical information we want for each indivudal
    statistics = tools.Statistics(lambda ind: ind.fitness.values[0])    # Get each ind's fitness value
    statistics.register("min", np.min)
    statistics.register("mean", np.mean)
    statistics.register("std", np.std)

    # Make a logbook for future reference
    logbook = tools.Logbook()
    logbook.header = ("gen", "nevals", "mean", "min", "std")

    # Fill in logbook w/ initial info
    stats = statistics.compile(pop)
    logbook.record(gen=0, nevals=len(pop), **stats)

    hall_of_fame.update(pop)
    # Evolution loop
    for gen in range(1, cfg.generations + 1):
        # Select offspring
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cfg.cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation
        for mutant in offspring:
            if random.random() < cfg.mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate individuals with invalid fitness
        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid)      # Produce new fitness values
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        # Elitism update if any new global best made from offspring
        hall_of_fame.update(offspring)

        # Inject hall of fame ind back into population
        offspring.sort(key=lambda ind: ind.fitness.values[0])
        for ind in range(cfg.eliteSize):
            offspring[ind] = toolbox.clone(hall_of_fame[ind])     
        
        pop[:] = offspring

        # Update Logbook for current generation
        stats = statistics.compile(pop)
        logbook.record(gen=gen, nevals=len(pop), **stats)

        # For console printing
        print(logbook.stream)
    
    best_ind = hall_of_fame[0]
    best_distance = best_ind.fitness.values[0]

    # Convert the indices into their city names
    tour_named = [cities[i] for i in best_ind]
    # Print start point for clarity
    tour_named.append(cities[best_ind[0]])

    return {
        "best_distance": best_distance,
        "best_tour_indices": best_ind,
        "tour_cities": tour_named,
        "logbook": logbook
    }
        

MAIN

In [13]:
def main():
    cfg = Config(seed=18)
    results = run(cfg)

    logbook = results["logbook"]    # access the logbook
    best_distance = results["best_distance"]
    best_ind = results["best_tour_indices"]
    cities_toured = results["tour_cities"]
    accuracy = compare_with_best(best_distance)
    print(logbook)

    print("\n===========================================")
    print("SHORTES PATH TOUR OBTAINED")
    print("===========================================")
    print(f"Best Minimum Distance Found = {best_distance:.4f} miles\n")
    print(f"Percentage difference from optimum solution: {accuracy:.4f}")
    
    print("Best Tour (Indices) Found:", best_ind)
    print("Best Tour (City names) Found:", cities_toured)



In [14]:
if __name__ == "__main__":
    main()

gen	nevals	mean   	min    	std    
0  	1500  	50649.3	38431  	3507.1 
1  	1500  	48056.4	36288.4	3156.43
2  	1500  	46181.9	36288.4	2921.75
3  	1500  	44499.8	35828  	2852.41
4  	1500  	43062.8	35499.6	2773.32
5  	1500  	41938.7	32418.3	2810.78
6  	1500  	40895  	29958.1	2776.84
7  	1500  	39826.4	29958.1	2894.01
8  	1500  	38766.3	28231.6	2705.89
9  	1500  	38005.5	28231.6	2761.74
10 	1500  	37260.8	27210.1	2793.73
11 	1500  	36371.4	27210.1	2709.29
12 	1500  	35546.5	27210.1	2722.29
13 	1500  	34856.8	27210.1	2637.41
14 	1500  	34300.1	27210.1	2770.11
15 	1500  	33561  	27210.1	2726.61
16 	1500  	32759.9	26311.2	2582.7 
17 	1500  	32181.1	25773.1	2454.36
18 	1500  	31702.8	24649.3	2451.75
19 	1500  	31130.4	24262.4	2462.45
20 	1500  	30659.2	24262.4	2394.44
21 	1500  	30215.2	24262.4	2300.57
22 	1500  	29985.6	23770.5	2411.89
23 	1500  	29456.9	22986.9	2321.34
24 	1500  	29144.7	22901.3	2407.01
25 	1500  	28682.4	18742.1	2314.91
26 	1500  	28396.2	18742.1	2374.84
27 	1500  	27905.2	1

Phase 1: Configuration search (operators + params)

Extended config and toolbox to try different crossover/mutation without changing existing code. Then run a small grid over operator combos and report the best.

In [22]:
# Extended config for search (operator choice + same GA params as Config)
@dataclass(frozen=True)
class ConfigSearch:
    genome_length: int = 49
    pop_size: int = 1500
    generations: int = 1000
    cxpb: float = 0.7
    mutpb: float = 0.19
    tournsize: int = 3
    eliteSize: int = 1
    seed: Optional[int] = None
    mutation_type: str = "inversion"   # "inversion" | "insert"
    crossover_type: str = "ordered"   # "ordered" | "pmx"
    # Phase 2: plateau escape (0 = disabled)
    plateau_threshold: int = 0       # gens without improvement before adaptive
    plateau_immigrant_frac: float = 0.03   # replace worst N% with random individuals
    plateau_mutpb_mult: float = 1.5   # multiply mutpb during adaptive burst (cap 0.4)
    plateau_tournsize_adaptive: int = 2   # lower selective pressure during burst
    plateau_cooldown: int = 80        # gens before allowing adaptive again


def build_toolbox_extended(cfg: ConfigSearch) -> base.Toolbox:
    ensure_deap_creators()
    toolbox = base.Toolbox()
    toolbox.register("indices", random.sample, range(cfg.genome_length), cfg.genome_length)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    if cfg.crossover_type == "pmx":
        toolbox.register("mate", tools.cxPartialyMatched)  # DEAP spelling
    else:
        toolbox.register("mate", tools.cxOrdered)

    if cfg.mutation_type == "insert":
        toolbox.register("mutate", mutInsert)
    else:
        toolbox.register("mutate", tools.mutInversion)

    toolbox.register("select", tools.selTournament, tournsize=cfg.tournsize)
    return toolbox

In [23]:
def run_extended(cfg: ConfigSearch, verbose: bool = False) -> Dict[str, Any]:
    """Evolution loop with build_toolbox_extended(cfg). Phase 2: plateau escape when plateau_threshold > 0."""
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)

    cities, coordinates = load_dat(FILENAME)
    Haversine_Dist_matrix = haversine_matrix(coordinates)

    toolbox = build_toolbox_extended(cfg)
    toolbox.register("evaluate", evaluate_TSP(Haversine_Dist_matrix))
    pop = toolbox.population(n=cfg.pop_size)

    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    hall_of_fame = tools.HallOfFame(cfg.eliteSize)
    statistics = tools.Statistics(lambda ind: ind.fitness.values[0])
    statistics.register("min", np.min)
    statistics.register("mean", np.mean)
    statistics.register("std", np.std)
    logbook = tools.Logbook()
    logbook.header = ("gen", "nevals", "mean", "min", "std")

    stats = statistics.compile(pop)
    logbook.record(gen=0, nevals=len(pop), **stats)
    hall_of_fame.update(pop)

    current_best = hall_of_fame[0].fitness.values[0]
    generations_without_improvement = 0
    adaptive_cooldown = 0

    for gen in range(1, cfg.generations + 1):
        use_adaptive = (
            cfg.plateau_threshold > 0
            and generations_without_improvement >= cfg.plateau_threshold
            and adaptive_cooldown == 0
        )

        if use_adaptive:
            tournsize = cfg.plateau_tournsize_adaptive
            mutpb_eff = min(cfg.mutpb * cfg.plateau_mutpb_mult, 0.4)
        else:
            tournsize = cfg.tournsize
            mutpb_eff = cfg.mutpb

        offspring = tools.selTournament(pop, len(pop), tournsize=tournsize)
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cfg.cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mutpb_eff:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid)
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        if use_adaptive and cfg.plateau_immigrant_frac > 0:
            n_immigrant = max(1, int(len(offspring) * cfg.plateau_immigrant_frac))
            offspring.sort(key=lambda ind: ind.fitness.values[0])
            for i in range(-n_immigrant, 0):
                offspring[i] = toolbox.clone(toolbox.individual())
                del offspring[i].fitness.values
            invalid = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(toolbox.evaluate, invalid)
            for ind, fit in zip(invalid, fitnesses):
                ind.fitness.values = fit

        hall_of_fame.update(offspring)
        offspring.sort(key=lambda ind: ind.fitness.values[0])
        for i in range(cfg.eliteSize):
            offspring[i] = toolbox.clone(hall_of_fame[i])
        pop[:] = offspring

        new_best = hall_of_fame[0].fitness.values[0]
        if new_best < current_best:
            current_best = new_best
            generations_without_improvement = 0
        else:
            if adaptive_cooldown == 0:
                generations_without_improvement += 1
        if use_adaptive:
            adaptive_cooldown = cfg.plateau_cooldown
        if adaptive_cooldown > 0:
            adaptive_cooldown -= 1

        stats = statistics.compile(pop)
        logbook.record(gen=gen, nevals=len(pop), **stats)
        if verbose:
            print(logbook.stream)

    best_ind = hall_of_fame[0]
    best_distance = best_ind.fitness.values[0]
    tour_named = [cities[i] for i in best_ind]
    tour_named.append(cities[best_ind[0]])

    return {
        "best_distance": best_distance,
        "best_tour_indices": best_ind,
        "tour_cities": tour_named,
        "logbook": logbook,
    }

In [24]:
# Phase 1 search: 4 operator combos x 2 param sets (current + higher mutation)
OPTIMUM = 10637.36
operator_combos = [
    ("inversion", "ordered"),
    ("inversion", "pmx"),
    ("insert", "ordered"),
    ("insert", "pmx"),
]
param_sets = [
    {"cxpb": 0.7, "mutpb": 0.19, "pop_size": 1500, "generations": 1000},
    {"cxpb": 0.7, "mutpb": 0.25, "pop_size": 1500, "generations": 1000},
]
seeds = [18, 42]

results = []
for (mut_type, cx_type) in operator_combos:
    for params in param_sets:
        for seed in seeds:
            cfg = ConfigSearch(
                mutation_type=mut_type,
                crossover_type=cx_type,
                seed=seed,
                **params,
            )
            res = run_extended(cfg, verbose=False)
            pct = compare_with_best(res["best_distance"])
            results.append({
                "mutation": mut_type,
                "crossover": cx_type,
                "mutpb": cfg.mutpb,
                "seed": seed,
                "best_distance": res["best_distance"],
                "pct_above_optimum": pct,
            })

best = min(results, key=lambda r: r["best_distance"])
print("Best configuration (Phase 1):")
print(f"  mutation={best['mutation']}, crossover={best['crossover']}, mutpb={best['mutpb']}, seed={best['seed']}")
print(f"  Best distance = {best['best_distance']:.4f} miles")
print(f"  % above optimum = {best['pct_above_optimum']:.4f}%")
print("\nAll runs (sorted by distance):")
for r in sorted(results, key=lambda x: x["best_distance"]):
    print(f"  {r['mutation']}+{r['crossover']} mutpb={r['mutpb']} seed={r['seed']}: {r['best_distance']:.2f} mi ({r['pct_above_optimum']:.2f}%)")

Best configuration (Phase 1):
  mutation=inversion, crossover=ordered, mutpb=0.25, seed=42
  Best distance = 10699.2230 miles
  % above optimum = 0.5816%

All runs (sorted by distance):
  inversion+ordered mutpb=0.25 seed=42: 10699.22 mi (0.58%)
  inversion+ordered mutpb=0.19 seed=18: 10766.47 mi (1.21%)
  inversion+ordered mutpb=0.19 seed=42: 10857.52 mi (2.07%)
  inversion+ordered mutpb=0.25 seed=18: 10873.23 mi (2.22%)
  insert+ordered mutpb=0.19 seed=18: 11025.02 mi (3.64%)
  insert+pmx mutpb=0.25 seed=42: 11074.84 mi (4.11%)
  inversion+pmx mutpb=0.19 seed=18: 11114.47 mi (4.49%)
  insert+ordered mutpb=0.19 seed=42: 11121.76 mi (4.55%)
  insert+ordered mutpb=0.25 seed=18: 11133.41 mi (4.66%)
  inversion+pmx mutpb=0.19 seed=42: 11265.82 mi (5.91%)
  inversion+pmx mutpb=0.25 seed=42: 11273.02 mi (5.98%)
  insert+ordered mutpb=0.25 seed=42: 11286.34 mi (6.10%)
  inversion+pmx mutpb=0.25 seed=18: 11323.45 mi (6.45%)
  insert+pmx mutpb=0.19 seed=18: 11329.50 mi (6.51%)
  insert+pmx mut

Phase 1 optimized

Focused search on the best combo (inversion + ordered): more mutation rates, more seeds, and longer runs to try to beat the Phase 1 best (10,699 mi).

In [25]:
# Phase 1 optimized: inversion+ordered only, more mutpb and seeds, then longer runs
OPTIMUM = 10637.36
phase1_opt_results = []

# Grid: mutpb around winning 0.25, more seeds
for mutpb in [0.22, 0.25, 0.28]:
    for seed in [18, 42, 100, 123, 256]:
        cfg = ConfigSearch(
            mutation_type="inversion",
            crossover_type="ordered",
            cxpb=0.7,
            mutpb=mutpb,
            pop_size=1500,
            generations=1000,
            seed=seed,
        )
        res = run_extended(cfg, verbose=False)
        pct = compare_with_best(res["best_distance"])
        phase1_opt_results.append({
            "mutpb": mutpb, "seed": seed, "generations": 1000,
            "best_distance": res["best_distance"], "pct_above_optimum": pct,
        })

# Longer runs (1500 gen) for best mutpb to try to push below 10699
for seed in [18, 42, 100]:
    cfg = ConfigSearch(
        mutation_type="inversion",
        crossover_type="ordered",
        cxpb=0.7,
        mutpb=0.25,
        pop_size=1500,
        generations=1500,
        seed=seed,
    )
    res = run_extended(cfg, verbose=False)
    pct = compare_with_best(res["best_distance"])
    phase1_opt_results.append({
        "mutpb": 0.25, "seed": seed, "generations": 1500,
        "best_distance": res["best_distance"], "pct_above_optimum": pct,
    })

best_opt = min(phase1_opt_results, key=lambda r: r["best_distance"])
print("Phase 1 optimized best:")
print(f"  mutpb={best_opt['mutpb']}, seed={best_opt['seed']}, gen={best_opt['generations']}")
print(f"  Best distance = {best_opt['best_distance']:.4f} miles")
print(f"  % above optimum = {best_opt['pct_above_optimum']:.4f}%")
print("\nTop 10 (sorted by distance):")
for r in sorted(phase1_opt_results, key=lambda x: x["best_distance"])[:10]:
    print(f"  mutpb={r['mutpb']} seed={r['seed']} gen={r['generations']}: {r['best_distance']:.2f} mi ({r['pct_above_optimum']:.2f}%)")

Phase 1 optimized best:
  mutpb=0.25, seed=42, gen=1000
  Best distance = 10699.2230 miles
  % above optimum = 0.5816%

Top 10 (sorted by distance):
  mutpb=0.25 seed=42 gen=1000: 10699.22 mi (0.58%)
  mutpb=0.25 seed=42 gen=1500: 10699.22 mi (0.58%)
  mutpb=0.28 seed=123 gen=1000: 10709.60 mi (0.68%)
  mutpb=0.28 seed=256 gen=1000: 10709.60 mi (0.68%)
  mutpb=0.22 seed=18 gen=1000: 10743.92 mi (1.00%)
  mutpb=0.28 seed=18 gen=1000: 10777.68 mi (1.32%)
  mutpb=0.25 seed=100 gen=1500: 10803.30 mi (1.56%)
  mutpb=0.25 seed=100 gen=1000: 10828.90 mi (1.80%)
  mutpb=0.25 seed=123 gen=1000: 10837.62 mi (1.88%)
  mutpb=0.28 seed=100 gen=1000: 10851.37 mi (2.01%)
